In [1]:
import networkx as nx # version 2.2
import numpy as np
import Cascade_generation_functions_NetRate as gen
import os
import glob

import sys
import time
import random

import Init_NetRate as Init
import cvxpy as cp
import Cascade_generation_functions_NetRate as Gen
import CVX_functions as cvx
import NetRate_ADMM_fcts as ADMM
import NetRate_SG_fcts as NetSG

import Init_NetInf
import Greedy_NetInf as Greed

In [2]:
def Global_Precision_Recall_Accuracy_MSE_NetRate(G_true,A_hat,lower_bound):
    correct = 0
    nb_edges = 0
    top_acc = 0
    bot_acc = 0
    for i in range(0,G_true.number_of_nodes()):
        for j in range(0,G_true.number_of_nodes()):
            if A_hat[i,j] >lower_bound:
                nb_edges +=1
                bot_acc +=1
                if (i,j) in G_true.edges():
                    correct +=1
                    bot_acc+=1
                else :
                    top_acc+=1
            else :
                if (i,j) in G_true.edges():
                    top_acc +=1
                    bot_acc+=1
                
    precision = correct/nb_edges
    recall = correct/G_true.number_of_edges()
    accuracy = 1-(top_acc/bot_acc)
    mse = 0
    nb_correcte_edges=0
    for edge in G_true.edges():
            try:
                true_alpha = G_true.edges[edge]["weight"][0]
            except TypeError:
                true_alpha = G_true.edges[edge]["weight"]
            approx_alpha = A_hat[edge[0],edge[1]]
            mse += pow((true_alpha-approx_alpha),2) # mean square error
    try :
        mse = mse/G_true.number_of_edges()
    except ZeroDivisionError:
        print ("There is no correct edge")
    return (precision,recall,accuracy,mse)

def Global_Precision_Recall_Accuracy_NetInf(G_true,G_max):
    correct = 0
    top_acc = 0
    bot_acc = 0
    nb_edges_G_max = G_max.number_of_edges()
    nb_edges_G_true = G_true.number_of_edges()
    for i in range(0,G_true.number_of_nodes()):
        for j in range(0,G_true.number_of_nodes()):
            if (i,j) in G_max.edges() :
                bot_acc +=1
                if(i,j) in G_true.edges() :
                    bot_acc +=1
                    correct+=1
                else :
                    top_acc +=1
            else :
                if (i,j) in G_true.edges():
                    top_acc +=1
                    bot_acc +=1
    precision = correct/nb_edges_G_max
    recall = correct/nb_edges_G_true
    accuracy = 1-top_acc/bot_acc
    return(precision,recall,accuracy)

def Write_info(file_path,time,algo_infos):
    precision,recall,accuracy,mse = algo_infos
    f = open(file_path,"w")
    f.write("General Infos\n")
    f.write("time needed : "+str(time)+" s\n")
    f.write("Accuracy of the algo : " +str(accuracy)+"\n")
    f.write("precision of the algo : "+str(precision)+"\n")
    f.write("recall of the algo : "+str(recall)+"\n")
    f.write("mean square error is : "+str(mse)+"\n")
    f.close()

In [10]:
'''
Global parameters for the cascade model and for all solving methods.
'''
window = 10
beta = 1 # used for the construction of the cascades, probability of the edge to transmit the infection
alpha_min = 0.0005
alpha_max = 10 #bound alpha from above
alpha_init = 0.01 #initial value of the infectious rates
tol = 0.05 # value from which we decide that there exist an edge
more_infos_Netrate = False # If set to true computes the time,accuracy and mean square error at each iteration (solves down the overall computartion)
# can also be modified for each solver.

'''For stochastic block model'''
nb_communities = 10 #number of communities we want our graph to have
p_1 = 0.4 # proba that the edge connect two vertices in the same community
p_2 = 0.15 # proba that the edge connect two vertices in different communities


solvers = ["ADMM"] #List containing the solver that should be used. "NetInf","CVX","SGD","ADMM"
gen = "Stochastic" # can either be "Erdos" or "Stochastic"
'''
NetRate with CVX
'''
more_infos_CVX = more_infos_Netrate

'''
NetRate with SGD parameters
'''
gamma_SGD = 0.0005
K = 15000 #number of itterations of SGD
more_infos_SGD = more_infos_Netrate

SGD_param = (alpha_init,alpha_max,K,gamma_SGD,alpha_min,window,tol,more_infos_SGD)

'''
Netrate with ADMM
'''
iter_ADMM = 50 #number of itteration for 1 node in the ADMM method. This is a parameter to tune
iter_GD_ADMM = 2000 #number of iterations of the gradient descent
gamma_ADMM = 0.00005 # Learning rate of the GD for alpha (maybe too small)
u = 4 # used for the gradient descent of rho and as a penalizer and the constrain
more_infos_ADMM = more_infos_Netrate 
param_ADMM = (alpha_init,alpha_max,window,iter_GD_ADMM,iter_ADMM,gamma_ADMM,u,alpha_min,tol,more_infos_ADMM)


iter_ADMM_SGD = 1500 # Test for the ADMM with SGD for the alpha update
gamma_ADMM_SGD = 0.0005
param_ADMM_SGD = (alpha_init,alpha_max,window,iter_GD_ADMM,iter_ADMM,gamma_ADMM_SGD,u,alpha_min,tol,more_infos_ADMM)

'''
NetInf paramters
'''
EPS = 1e-64 #zero machine
ALPHA = 1.0 #Incubation parameter (for exp and power law)
MODEL = 0 # 0 = exp law, 1 = power law (power law is not  implemented yet) When constructing the underlying network specify that we use the exponential law
MAX = sys.float_info.max #Max value of a float in python
#MIN = sys.float_info.min #Min value of a float in python
MIN = -MAX
#(works only if groundtruth is available)
#When set to True (especially boundOn) it slow down greatly the computation
compare_groud_truth = False # If set to True outputs some aditional information (precision and recall of the algo)
boundOn = False

greedy_global_param = (ALPHA,MODEL,MAX,MIN,EPS,compare_groud_truth,boundOn)


In [4]:
cascade_path ="bla.txt"
G_true = Gen.Generate_random_graph(50,100)
Cascades = Gen.Generate_all_cascades(G_true,-300,window,MODEL,beta)
Gen.Save_cascade_to_file(cascade_path,Cascades,G_true)
G_star,C_dic = Init.Init(cascade_path)
N = G_true.number_of_nodes()

All nodes were read


In [5]:
t_s = time.time()
A_hat_ADMM,obj_per_iter_per_node_ADMM,acc_mse_time_ADMM = ADMM.NetRate_ADMM(G_true,C_dic,N,param_ADMM)
t_f = time.time()
print(t_f-t_s)
algo_infos_ADMM = Global_Precision_Recall_Accuracy_MSE_NetRate(G_true,A_hat_ADMM,0.05)
print(algo_infos_ADMM)

68.51080632209778
(0.9270833333333334, 0.89, 0.9081632653061225, 0.02387663973705827)


In [9]:
t_s = time.time()
A_hat_ADMM,obj_per_iter_per_node_ADMM,acc_mse_time_ADMM = ADMM.NetRate_ADMM_SGD(G_true,C_dic,N,param_ADMM,iter_ADMM_SGD)
t_f = time.time()
print(t_f-t_s)
algo_infos_ADMM = Global_Precision_Recall_Accuracy_MSE_NetRate(G_true,A_hat_ADMM,0.05)
print(algo_infos_ADMM)

6.204376220703125


ZeroDivisionError: division by zero